In [1]:
# !pip install cx_Oracle
# import cx_Oracle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# If you are using Wallet file, provide the zip file path for `wallet_location`
connection_parameters = {
    "user_name": "admin",
    "password": "Oracle_12345",
    "service_name": "adw_low",
    "wallet_location": "Wallet_ADW.zip"
}
import pandas as pd
import ads



In [ ]:
# simple read of a SQL query into a dataframe with no bind variables
ride_df = pd.DataFrame.ads.read_sql(
    "SELECT * FROM TRIP_DATA",
    connection_parameters=connection_parameters,
)

In [2]:
cus_df=pd.read_csv('Cust_data.csv')
ride_df=pd.read_csv('Trip_data.csv')
driver_df=pd.read_csv('Driver_data.csv')

In [8]:
cus_df.drop('Unnamed: 0',axis=1,inplace=True)
driver_df.drop('Unnamed: 0',axis=1,inplace=True)
ride_df.drop('Unnamed: 0',axis=1,inplace=True)

In [9]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the  distance between two points on a Earth."""


    # Convert latitude and longitude from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    # Radius of the Earth in kilometers 
    r = 6371

    # Calculate the distance
    distance = r * c
    return distance

In [10]:
#  DataFrame named 'ride_df' with columns 'latitude' and 'longitude'
ride_df['distance_km'] = ride_df.apply(lambda row: haversine_distance(row['Origin_latitude'], row['Origin_longitude'], row['Dest_latitude'], row['Dest_longitude']), axis=1)


In [11]:
# Convert the 'datetime_column' to datetime format
ride_df['Trip_StartTime'] = pd.to_datetime(ride_df['Trip_StartTime'])

# Create new columns for date and time (without UTC)
ride_df['Trip_StartTime'] = ride_df['Trip_StartTime'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [12]:
ride_df=ride_df.sort_values(by='Trip_StartTime')

In [13]:
ride_df=ride_df.reset_index(drop=True)

In [14]:
# Convert 'pickup_datetime' to datetime format (skip this step if already in datetime format)
ride_df['Trip_StartTime'] = pd.to_datetime(ride_df['Trip_StartTime'])

In [15]:
def remove_outliers_iqr(series,q3):
    """Removes outliers from a pandas Series using the interquartile range (IQR) method.

    Args:
        series: A pandas Series.
        q3 : 75 percentile.

    Returns:
        A pandas Series with outliers removed.
    """
    # Calculate the first and third quartiles (Q1 and Q3) of the Series
    q1 = series.quantile(0.25)
    q3 = series.quantile(q3)
    
    # Calculate the interquartile range (IQR)
    iqr = q3 - q1
    
    # Define the lower and upper bounds for outliers
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    # Remove values outside the lower and upper bounds
    series_outliers_removed = series[(series >= lower_bound) & (series <= upper_bound)]
    
    
    return series_outliers_removed


In [16]:
ride_df['fare_amount']=remove_outliers_iqr(ride_df['fare_amount'],0.90)

In [17]:
# fill nan by median
ride_df['fare_amount'].fillna(ride_df['fare_amount'].median(),inplace=True)

In [18]:
ride_df.loc[ride_df['distance_km']>2000,'distance_km']=ride_df['distance_km'].quantile(0.75)

In [19]:
ride_df.loc[ride_df.distance_km==0,['fare_amount']]=0

In [20]:
ride_df.loc[(ride_df.distance_km >80) & (ride_df.fare_amount ),['fare_amount']]=49

In [21]:
ride_df.loc[(ride_df.distance_km < 1) & (ride_df.fare_amount >30 ),['fare_amount']] = ride_df.fare_amount.quantile(0.25)

In [22]:
driver_df['Driver_Age_range']=pd.cut(driver_df.Age,bins=[26,33,40,45,60],labels=['Young','Adult','Elder','Old age'])

In [23]:
ride_df['distance_km_range']=pd.cut(ride_df.distance_km,bins=[-1,2,4,8,127],labels=['very_short','short','long','very_long'])

In [24]:
ride_df=ride_df.loc[~(ride_df.Origin_longitude <= -91)]

In [25]:
ride_df=ride_df.loc[ride_df['distance_km'] < 100]

In [26]:
ride_df.reset_index(drop=True,inplace=True)

In [27]:
s=ride_df.loc[ride_df.distance_km!=0].index.to_list()


In [28]:
for i in ride_df.iloc[s].index:
    ride_df.iloc[i]['distance_km'] = np.random.randint(0,3) + ride_df.iloc[i]['distance_km']
    



/tmp/ipykernel_6262/2258168249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ride_df.iloc[i]['distance_km'] = np.random.randint(0,3) + ride_df.iloc[i]['distance_km']


In [29]:
ride_df['hours_range']=pd.cut(ride_df.hour,bins=[-1,3,6,8,11,14,17,21,24],labels=['12:00AM-03:00AM','03:00AM-06:00AM','06:00AM-09:00AM','09:00AM-12:00PM','12:00PM-03:00PM','03:00PM-06:00PM','06:00PM-09:00PM','12:00AM-3:00AM'])
# pd.cut(driver_df.Age,bins=[26,33,40,45,60],labels=['Young','Adult','Elder','Old age'])

In [31]:
ride_df['fare_amount_range']=pd.cut(ride_df.fare_amount,bins=[-1,5,20,40,500],labels=['Inexpensive','Affordable','Economical','Expensive'])

In [32]:
ride_df.value_counts()

Trip_ID   fare_amount  Trip_StartTime       Origin_longitude  Origin_latitude  Dest_longitude  Dest_latitude  passenger_count  Customer_ID  Driver_ID  driver_rated_for_trip  customer_rated_for_trip  Trip_cancellation  Cancelled_by  Reason                          year  month  date  hour  minutes  seconds  Region            distance_km  distance_km_range  hours_range      fare_amount_range
6674      6.5          2015-01-12 17:01:10  -73.989960        40.757118        -73.978287      40.760780      1                C38493       D13828     3                      5                        0                  0             0                               2015  1      12    17    1        10       Pubs              1.064161     very_short         03:00PM-06:00PM  Affordable           1
36987373  6.0          2015-02-25 08:45:13  -73.983276        40.734722        -73.997002      40.728813      4                C25673       D14244     2                      3                        0           

In [31]:
ride_df.to_csv('ride_df_prepro_1.csv')